## data preprocess
### data cleaning on quality issues
For each feature identify the following problems
* Inconsistent values
* Duplicate records
* Missing values
* Invalid values
* Outliers

### data transformation
scaling, transformation, feature selection, dim reduction, data manipulation
* scaling - normalization
* transformation - category vectorize
* feature selection/engineering

In [3]:
import pandas as pd
import sys
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
color = sns.color_palette()

ServerName = "localhost"
Database = "sdra"

# Create the connection
engine = create_engine('postgresql://postgres:@' + ServerName + ':5432/' + Database)

### check address data
* missing data(addresses)?

In [3]:
sql = '''
with properties as (
    select number,upper(street) as street, city,zip from redfin_properties
)
-- select number,street,city from properties where (number, street,city) not in (select number,street,city from addresses)
-- select p.number,p.street,p.city from properties p,addresses a
-- select count(1) from properties p left join addresses a on p.number=a.number and a.street = p.street and p.city=a.city and p.zip = a.postcode
select count(1) from properties p where (p.number, p.street, p.city, p.zip) in (select a.number, a.street, a.city,a.postcode as zip from addresses a)
'''

valid_count = pd.read_sql_query(sql, engine)

In [4]:
valid_count

,count
0,524093


### redfin_properties
* (invalid) year_built: eg 9999
* (missing) sqft

### redfin_transactions
* (missing value) transactions missing between 1980~1987
* (mismatch) sold price mismatch btw 'public record and sandicor': https://www.redfin.com//CA/Imperial-Beach/100-Evergreen-Ave-91932/unit-2/home/143020730
* (note) price - latest sold or listing price
  * https://www.redfin.com//CA/Imperial-Beach/100-Evergreen-Ave-91932/unit-2/home/143020730
  * https://www.redfin.com//CA/San-Diego/10260-Black-Mountain-Rd-92126/unit-135/home/4855426
* outlier: unit price(price_per_sqft) > $1500
  